In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pytorch-tabnet transformers

import pandas as pd
import shutil
import os
import numpy as np
from transformers import BertTokenizer
from transformers import BertModel
import torch
import random
import pickle

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns

     |████████████████████████████████| 1.5MB 21.7MB/s 
     |████████████████████████████████| 890kB 50.5MB/s 
     |████████████████████████████████| 2.9MB 51.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=bc5ad5e31d402665354bee445faf9f13af459ab4d9dcdf943e4ed7dbaa86a266
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### 1. Загрузка необходимых датасетов

In [72]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/train.csv', sep=';')
ed = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/education.csv', sep=';')
ep = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/employements.csv', sep=';')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/test.csv', sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


####*1.1 Подгрузим дополнительные данные о зраплатах в России по городам. Возьмем рейтинг городов России по средневзвешенной зарплате  1-ое полугодие 2020-го года*

In [73]:
df_salary = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/salary_by_city - salary_by_city - Лист1.csv')
df_salary['locality_name'] = df_salary['location']
df_salary['salary_count'] = df_salary['salary']
df_1 = pd.merge(df, df_salary, how='left', on="locality_name")


In [133]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306270 entries, 0 to 306269
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id                          306270 non-null  int64  
 1   position_x                  306270 non-null  object 
 2   region                      306258 non-null  object 
 3   industry                    306270 non-null  object 
 4   locality                    306270 non-null  int64  
 5   locality_name               290778 non-null  object 
 6   education_type              306270 non-null  object 
 7   drive_licences              121925 non-null  object 
 8   citizenship                 306270 non-null  object 
 9   schedule                    306270 non-null  object 
 10  employement_type            306270 non-null  object 
 11  age                         298459 non-null  float64
 12  gender                      298497 non-null  object 
 13  experience    

In [76]:
#Заполним недостающие значения по городам самым низким показателем зарплат
df_1.salary_count.fillna('28500')

0         39800
1         34000
2         28500
3         35900
4         46400
          ...  
306265    28500
306266    28500
306267    28500
306268    30600
306269    33200
Name: salary_count, Length: 306270, dtype: object

In [77]:
# очистим собранные средние зарплаты по городам от недостающих значений

df_1['salary_count'].isna().sum()

sd_median=df_1['salary_count'].median(skipna = bool)
df_1['salary_count']= pd.to_numeric(df_1['salary_count'],errors='coerce')
df_1['salary_count']= np.where(np.isnan(pd.to_numeric(df_1['salary_count'],errors='coerce')),sd_median,df_1['salary_count'])

sd_median=df_1['salary_count'].median(skipna = bool)
df_1['salary_count']= pd.to_numeric(df_1['salary_count'],errors='coerce')
df_1['salary_count']= np.where(np.isnan(pd.to_numeric(df_1['salary_count'],errors='coerce')),sd_median,df_1['salary_count'])

df_1['salary_count'].isna().sum()

172864

###2.Кластеризуем каждое резюме по уровню средней зарплаты, характерной для города или региона

In [81]:
#скалируем данные по зарплатам
from sklearn.preprocessing import StandardScaler
import numpy as np
X=df_1.iloc[:, 27].values.reshape(-1, 1)
X=np.nan_to_num(X)
Clus_dataSet=StandardScaler().fit_transform(X)

In [82]:
X=df_1.iloc[:, 27].values
print(X)

[39800. 34000. 43800. ... 43800. 30600. 33200.]


In [83]:
#кластеризуем с помощью  Kmeans
from sklearn.cluster import KMeans
clasterNum=10
k_maens = KMeans(init="k-means++",n_clusters=clasterNum,n_init=12)
k_maens.fit(Clus_dataSet)
labels=k_maens.labels_

In [84]:
#вот и появился новый признак - кластер, который агрегирует в себе данные по локали 
#и общем уровне зарплат, характерном для этой локации

df_1['cluster'] = labels

In [85]:
#подгрузим датасет о предыдушем опыте работника, где нас особенно интерсует графа с описанием его обязанностей

df_1 = pd.merge(df_1, ep, how='left', on="id")

In [87]:
#почистим признак от шума

df_1.responsibilities = df_1.responsibilities.str.replace(r'<(.*?)>', '')
df_1.responsibilities = df_1.responsibilities.str.replace('/', '')
df_1.responsibilities = df_1.responsibilities.str.replace(',', '')
df_1.responsibilities = df_1.responsibilities.str.replace(' +', ' ')
df_1.responsibilities = df_1.responsibilities.str.strip()

#почистим сведения об обязанностях на предыдущем месте и работы и типе образования от нулевых значений

df_1['responsibilities']= np.where( pd.isnull(df_1['responsibilities']),'no responsibilities',df_1['responsibilities'])

df_1['education_type']= np.where( pd.isnull(df_1['education_type']),'no education type',df_1['education_type']) 

In [93]:
df_work = df_1[['id', 'industry', 'employement_type',
              'education_type', 'salary_x', 'cluster', 'responsibilities']]

###3. Обучим на Bert векторные представления признака 'responsibilities'

In [99]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [100]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
bert = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

In [101]:
#df_work[['responsibilities']] = df_work[['responsibilities']].astype(str)
df_work.loc[:, ('responsibilities')] = df_work.loc[:, ('responsibilities')].astype(str)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [102]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306270 entries, 0 to 306269
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                306270 non-null  int64 
 1   industry          306270 non-null  object
 2   employement_type  306270 non-null  object
 3   education_type    306270 non-null  object
 4   salary_x          306270 non-null  int64 
 5   cluster           306270 non-null  int32 
 6   responsibilities  306270 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 17.5+ MB


In [103]:
%%time

bert.to(device)
bert.eval();

step = 10_000
responsibilities_vectors = False

for batch_i in range(0, len(df_work), step):
    batch = df_work.responsibilities[batch_i:batch_i+step]
    inputs = tokenizer(
        batch.to_list(),
        padding='max_length',
        truncation=True,
        max_length=20,
        return_tensors='pt',
        return_attention_mask=True,
    )
    inputs.to(device)
    with torch.no_grad():
        output = bert(**inputs)
    total_sum = torch.zeros_like(output[0][:, 1, :])
    for i in range(0, output[0].shape[1]):
        total_sum += output[0][:, i, :]
    total_sum = total_sum / output[0].shape[1]
    # responsibilities_vectors.extend(total_sum.detach().cpu().numpy())
    total_sum = total_sum.detach().cpu().numpy()
    if responsibilities_vectors is False:
        responsibilities_vectors = total_sum
    else:
        responsibilities_vectors = np.vstack((responsibilities_vectors, total_sum))
    
    if batch_i % 100_000 == 0:
        print(batch_i)

0
100000
200000
300000
CPU times: user 7min 47s, sys: 3min 31s, total: 11min 18s
Wall time: 11min 18s


In [132]:
len(responsibilities_vectors[6])


768

In [122]:
responsibilities_vectors.shape

(306270, 768)

In [108]:
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/responsibilities_vectors.npy', responsibilities_vectors)

###4. Преобразуем в индексы другие признаки

In [109]:
cluster = set()
education_type = set()
employement_type = set()
industry = set()

cluster.update(df_work.cluster.unique().tolist())
education_type.update(df_work.education_type.unique().tolist())
employement_type.update(df_work.employement_type.unique().tolist())
industry.update(df_work.industry.unique().tolist())

In [111]:
cluster2id = {name:id for id, name in enumerate(cluster)}
education_type2id = {name:id for id, name in enumerate(education_type)}
employement_type2id = {name:id for id, name in enumerate(employement_type)}
industry2id = {name:id for id, name in enumerate(industry)}

In [113]:
df_work.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306270 entries, 0 to 306269
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                306270 non-null  int64 
 1   industry          306270 non-null  object
 2   employement_type  306270 non-null  object
 3   education_type    306270 non-null  object
 4   salary_x          306270 non-null  int64 
 5   cluster           306270 non-null  int32 
 6   responsibilities  306270 non-null  object
dtypes: int32(1), int64(2), object(4)
memory usage: 17.5+ MB


In [114]:
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/cluster2id.pickle', 'wb') as handle:
    pickle.dump(cluster2id, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/education_type2id.pickle', 'wb') as handle:
    pickle.dump(education_type2id, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/employement_type2id.pickle', 'wb') as handle:
    pickle.dump(employement_type2id, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/industry2id.pickle', 'wb') as handle:
    pickle.dump(industry2id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [115]:
cluster_label = []
education_type_label = []
employement_type_label = []
industry_label = []
salary_from	= []

for index in df_work.index:
    cluster_label.append(cluster2id[df_work.cluster.iloc[index]])
    education_type_label.append(education_type2id[df_work.education_type.iloc[index]])
    employement_type_label.append(employement_type2id[df_work.employement_type.iloc[index]])
    industry_label.append(industry2id[df_work.industry.iloc[index]])
    salary_from.append(df_work.salary_x.iloc[index])

###5.Cохраним наши индексы отдельно в файле

In [116]:

salary_from = np.array(salary_from)
salary_from_scaled = preprocessing.scale(salary_from)

In [117]:
salary_from_scaled[22], salary_from[22]

(-0.5334260573415979, 22080)

In [118]:
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/salary_scaled.npy', salary_from_scaled)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/salary_orig.npy', salary_from)

In [119]:
cluster_label = np.array(cluster_label)
education_type_label = np.array(education_type_label)
employement_type_label = np.array(employement_type_label)
industry_label = np.array(industry_label)

In [120]:
cluster_label = np.reshape(cluster_label, (cluster_label.shape[0], 1))
education_type_label = np.reshape(education_type_label, (education_type_label.shape[0], 1))
employement_type_label = np.reshape(employement_type_label, (employement_type_label.shape[0], 1))
industry_label = np.reshape(industry_label, (industry_label.shape[0], 1))

In [121]:
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/cluster_label.npy', cluster_label)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/education_type_label.npy', education_type_label)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/employement_type_label.npy', employement_type_label)
np.save('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/industry_label.npy', industry_label)